In [ ]:
# do history matching on the original stage 5/6/7 to match the wave front
# Then I can do the simulation
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
# 1. do the history matching for the stage 7 and stage 8
# Load the data from stage 7
datapath = "./"
import os

# Stage 7 DAS/gauge data history matching

## Plot the LF-DAS/gauge data at stage 7/S well

In [12]:
# Load DAS data
stage = 7 

# file path
DASdata_filepath = datapath + f"s_well/DASdata/LFDASdata_stg{stage}_swell.npz"
gauge_data_folder = datapath + "s_well/gauge_data/"
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage{stage}.npz"

# Load the geometry
gauge_md_geometry = datapath + "s_well/geometry/gauge_md_swell.npz"
frac_hit_geometry = datapath + f"s_well/geometry/frac_hit/frac_hit_stage_{stage}_swell.npz"

In [13]:
# Load the data
from DSS_analyzer_Mariner import Data1D_GAUGE, Data2D_XT_DSS, Data3D_geometry, Data1D_PumpingCurve

pumping_curve_dataframe = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)
gauge_md_geometry_dataframe = Data3D_geometry.Data3D_geometry(gauge_md_geometry)
frac_hit_geometry_dataframe = Data3D_geometry.Data3D_geometry(frac_hit_geometry)
DASdata = Data2D_XT_DSS.Data2D()
DASdata.loadnpz(DASdata_filepath)